In [1]:
import numpy as np
import pandas as pd
import re
from sklearn import preprocessing
from scipy.sparse import csr_matrix

In [2]:
states = ["INITIAL","login","View_Items","home","logout","View_Items_quantity","Add_to_Cart","shoppingcart",
          "remove","deferorder","purchasecart","inventory","sellinventory","clearcart","cancelorder"]
sessions = {'HZKS0-WG8pZr0eCsZlBAP5Xm': ['INITIAL','login',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'View_Items',
   'home',
   'logout','$'],'5tPgZbHdK2Zp+heFBs8HsMkx': ['INITIAL','login',
   'View_Items_quantity',
   'Add_to_Cart',
   'View_Items_quantity',
   'Add_to_Cart',
   'View_Items_quantity',
   'Add_to_Cart',
   'View_Items_quantity',
   'Add_to_Cart',
   'View_Items_quantity',
   'Add_to_Cart',
   'shoppingcart',
   'remove',
   'shoppingcart',
   'remove',
   'shoppingcart',
   'remove',
   'shoppingcart',
   'remove',
   'deferorder',
   'home',
   'logout','$']}

In [47]:
def transition_matrix(sessions, states):
    markovchains = []
    for key, value in sessions.items():
        # labelEncoding
        le = preprocessing.LabelEncoder()
        le.fit(value)
        transformed_s = le.transform(value)
        
        #factorize
        factorize = pd.factorize(value)[0]
        
        # matrix
        n = 1 + max(factorize)  # number of states
        M = [[0] * n for _ in range(n)]

        for (i, j) in zip(factorize, factorize[1:]):
            M[i][j] += 1
        
        # now convert to probabilities:
        for row in M:
            s = sum(row)
            if s > 0:
                row[:] = [f / s for f in row]
                
        # print Matrix style
        #for row in M: print(' '.join('{0:.2f}'.format(x) for x in row))
        
        # unique array in the right order
        value = np.array(value)
        _, idx = np.unique(value, return_index=True)
        
        df = pd.DataFrame(data = M, index=value[np.sort(idx)],
                          columns=value[np.sort(idx)])
        df_1 = pd.DataFrame(index=states, columns=states, dtype='float64')
        merge = df_1.merge(df, how='right').fillna(0).round(2).set_index(value[np.sort(idx)])

        # csr sparse matrix
        merge = csr_matrix(merge)
        markovchains.append(merge)
        
        
    return merge

m = transition_matrix(sessions, states)
